# <p style="text-align: center;"> Variational Linear Systems: Simple Example </p> 
<p style="text-align: center;"> Ryan LaRose </p>

This notebook briefly demonstrates the current state of the Variational Linear Systems (VLS) code. All code is contained in `vls_pauli.py`, which defines a `PauliSystem` class.

In [1]:
# =======
# imports
# =======
import time

import numpy as np

from vls_pauli import PauliSystem
from cirq import ParamResolver, Symbol, ops, Circuit, LineQubit

# Creating a Linear System of Equations

A `PauliSystem` consists of a matrix of the form
\begin{equation}
A = \sum_{k = 1}^{K} c_k \sigma_k
\end{equation}
where $c_k$ are complex coefficients and $\sigma_k$ are strings of Pauli operators. In code, we represent the matrix $A$ as arrays of strings corresponding to Pauli operators. For example, to represent the Pauli operators
\begin{align}
\sigma_1 &= \sigma_I \otimes \sigma_x \otimes \sigma_Y \otimes \sigma_Z
\end{align}
we would write:

In [2]:
# specify the pauli operators of the matrix
Amat_ops = np.array([["I", "X", "Y", "Z"]])

To store more terms, we simply append more lists of Pauli operators (string keys) to the operator matrix above. Coefficients $c_k$ are stored similarly as arrays of complex values:

In [3]:
# specify the coefficients of each term in the matrix
Amat_coeffs = np.array([1-0j])

Finally, the solution vector
\begin{equation}
|b\rangle = U |0\rangle
\end{equation}
is represented by the unitary $U$ that (efficiently) prepares $|b\rangle$ from the ground state. For example, the unitary $U$ could be
\begin{equation}
U = \sigma_I \otimes \sigma_X \otimes \sigma_Y \otimes \sigma_Z,
\end{equation}
which we would represent in code as:

In [4]:
# specify the unitary that prepares the solution vector b
Umat_ops = np.array(["I", "X", "Y", "Z"])

To create `PauliSystem`, we can then simply feed in `Amat_coeffs`, `Amat_ops`, and `Umat_ops`.

In [5]:
# create a linear system of equations
system = PauliSystem(Amat_coeffs, Amat_ops, Umat_ops)

# Working with a `PauliSystem`

The `PauliSystem` class can tell basic information about the system:

In [6]:
print("Number of qubits in system:", system.num_qubits())
print("Size of matrix:", system.size())

Number of qubits in system: 4
Size of matrix: (16, 16)


To see the actual matrix representation of the system (in the computational basis), we can do:

In [7]:
# get the matrix of the system
matrix = system.matrix()
print(matrix)

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j

We can also see the solution vector $|b\rangle$ by doing:

In [8]:
b = system.vector()
print(b)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+1.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]


# Creating an Ansatz

Initially, the `PauliSystem` ansatz for $V$ is an empty circuit:

In [9]:
# print out the initial (empty) ansatz
system.ansatz

We are free to pick whatever ansatz we wish. Here, we will start with the two-qubit alternating ansatz and simplify it to single qubit rotations. The two-qubit alternating ansatz is built-in to the `PauliSystem` class and can be easily created by doing:

In [10]:
system.make_ansatz_circuit()
system.ansatz

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────@───X^Symbol("39")───Y^Symbol("40")───Z^Symbol("41")───────X───X^Symbol("45")───Y^Symbol("46")───Z^Symbol("47")───────X───
                                                         │                                                      │                                                          │                                                          │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───X───X^Symbol("24")───Y^Symbol("25")───Z^Symbol("26")───@───┼───X^Symbol("30")───Y^Symbol("31")───Z^Symbol("32")───@───┼───
                                                                                                                                                                       │   │                                                      │   │
3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───@───X^Symbol("27")───Y^Symbol("28")───Z^Symbol("29")───X───┼───X^Symbol("33")───Y^Symbol("34")───Z^Symbol("35")───X───┼───
                                                         │                                                      │                                                          │                                                          │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───X───X^Symbol("36")───Y^Symbol("37")───Z^Symbol("38")───────@───X^Symbol("42")───Y^Symbol("43")───Z^Symbol("44")───────@───

This circuit contains 48 parameters (4 qubits x 2 "gates" / qubit x 6 parameters / gate). (Note that printing the circuit gets cut off in the notebook, scroll side to side to see the entire circuit.) For our simple example, we will chop off some of the gates to make the optimization easier:

In [11]:
# remove some of the gates and print it out
system.ansatz = system.ansatz[:-13]
system.ansatz

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────

2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────

3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───

4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───

# Computing the Cost

The local cost function is computed via the Hadamard Test. The local cost function can be written
\begin{equation}
C_1 = 1 - \frac{1}{n} \sum_{k = 1}^{K} \sum_{l \geq k}^{K} \frac{w_{k, l} c_k c_l^*}{\langle 0 | V^\dagger A_k^\dagger A_l V | 0 \rangle} \sum_{j = 1}^{n} \text{Re} \, \langle V_{k, l}^{(j)} \rangle
\end{equation}
where
\begin{equation}
\langle V_{k, l}^{(j)} \rangle := \langle0^{\otimes n}| V^\dagger A_k^\dagger U P_j U^\dagger A_l V |0^{\otimes n}\rangle
\end{equation}
and 
\begin{equation}
    w_{k, l} = \begin{cases}
    1 \qquad \text{if } k = l\\
    2 \qquad \text{otherwise}
    \end{cases} .
\end{equation}
Thus we have $(n + 1) K^2$ different circuits to run in order to compute the cost. For this simple example, $n = 4$ and $K = 1$, so we only have five circuits to run. The circuit for computing $\langle V_{1, 1}^{(1)} \rangle$ is shown below:

In [12]:
system.make_hadamard_test_circuit(system.ops[0], system.ops[0], 0, "real")

0: ───H──────────────────────────────────────────────────────@───@───@───@───────@───@───@───H───M('z')───
                                                             │   │   │   │       │   │   │
1: ───────X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────┼───┼───┼───@───────┼───┼───┼────────────────
                                                             │   │   │           │   │   │
2: ───────X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───┼───┼───X───X───X───┼───┼────────────────
                                                                 │   │               │   │
3: ───────X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───────Y───┼───Y───Y───────Y───┼────────────────
                                                                     │                   │
4: ───────X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───────────Z───Z───Z───────────Z────────────────

The circuit for computing the norm
\begin{equation}
\langle 0 | V^\dagger A_k^\dagger A_l V | 0 \rangle = \langle \psi | A_k^\dagger A_l | \psi \rangle 
\end{equation}
for the example $k = 0$, $l = 0$ is shown below:

In [13]:
system.make_norm_circuit(system.ops[0], system.ops[0], "real")

0: ───H──────────────────────────────────────────────────────@───@───@───@───@───@───H───M('z')───
                                                             │   │   │   │   │   │
1: ───────X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────┼───┼───┼───┼───┼───┼────────────────
                                                             │   │   │   │   │   │
2: ───────X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───┼───┼───X───┼───┼────────────────
                                                                 │   │       │   │
3: ───────X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───────Y───┼───────Y───┼────────────────
                                                                     │           │
4: ───────X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───────────Z───────────Z────────────────

To compute the cost, we can call `PauliSystem.cost` or `PauliSystem.eff_cost` (the latter exploits symmetries to compute the cost more efficiently) and pass in a set of angles to the ansatz gates:

In [17]:
# =======================================
# compute the cost for some set of angles
# =======================================

# normalize the coefficients
system.normalize_coeffs()

# get some angles
angles = np.random.randn(18)

# compute the cost and time it
start = time.time()
cost = system.eff_cost(angles)
end = time.time() - start

# print out the results
print("Local cost C_1 =", cost)
print("Time to compute cost =", end, "seconds")

0.93245
Local cost C_1 = 0.93245
Time to compute cost = 0.28073954582214355 seconds


# Solving the System

To solve the system, we minimize the cost function. We'll do this below with the Powell optimization algorithm.

In [21]:
# ===============================================
# minimize the cost (prints each cost evaluation)
# ===============================================
start = time.time()
out = system.solve(x0=angles, opt_method="COBYLA")
end = time.time() - start

0.9289499999999999
0.5724
0.9228500000000001
0.5689500000000001
0.60015
0.59275
0.5699000000000001
0.5736
0.5724
0.5871
0.57345
0.58565
0.56935
0.8321000000000001
0.8319
0.5731999999999999
0.78175
0.7933
0.5726
1.1881
0.90985
0.7093499999999999
0.68825
0.5679
0.486
0.46304999999999996
0.5261
0.4115
0.40464999999999995
0.40035
0.32825000000000004
0.21225000000000005
0.17710000000000004
0.1372
0.1362
0.12850000000000006
0.1290499999999999
0.14834999999999998
0.13065000000000004
0.1499999999999999
0.13560000000000005
0.15510000000000002
0.13044999999999995
0.15080000000000005
0.18009999999999993
0.1735
0.07740000000000002
0.08599999999999997
0.11825000000000008
0.14174999999999993
0.07645000000000002
0.06230000000000002
0.06555
0.052000000000000046
0.05315000000000003
0.11620000000000008
0.05710000000000004
0.054750000000000076
0.05630000000000002
0.05095000000000005
0.0616000000000001
0.05689999999999995
0.05625000000000002
0.041749999999999954
0.06655
0.03964999999999996
0.0422000000000

In [23]:
print("It took {} minute(s) to solve the system.".format(round(end / 60)))
#print("Number of iterations of optimization method:", out["nit"])
print("Number of function evaluations:", out["nfev"])

It took 1 minute(s) to solve the system.
Number of function evaluations: 227


# Comparing the Estimated and Exact Solutions

Below we print out the cost at the optimal angles found for the ansatz and print out the ansatz circuit with the optimal angles.

In [24]:
# get the optimal angles
opt_angles = out["x"]

# evaluate the cost at the optimal angles found
system.eff_cost(opt_angles)

# get a param resolver
param_resolver = ParamResolver(
    {str(ii) : opt_angles[ii] for ii in range(len(opt_angles))}
)

sol_circ = system.ansatz.with_parameters_resolved_by(param_resolver)
sol_circ

0.00040000000000006697


1: ───X^-0.994─────Y^-0.991─────Z^0.915────

2: ───X^-0.00312───Y^0.0113─────Z^-0.588───

3: ───X^0.0075─────Y^0.000161───Z^0.944────

4: ───X^0.00105────Y^-0.00422───Z^-0.38────

Next we convert the circuit to a unitary matrix and get the first column to check our solution with the actual solution.

In [25]:
# get the approximate solution and compute "bhat"
xhat = sol_circ.to_unitary_matrix()[:, 0]
bhat = np.dot(matrix, xhat)

# print out the overlap between "bhat" and the actual solution vector b
print("overlap of computed and exact solution =", np.dot(b.conj().T, bhat))

overlap of computed and exact solution = (0.04257143659529625-0.9986949908283624j)


In [29]:
# make sure both vectors are normalized
print("<bhat|bhat> =", np.dot(bhat.conj().T, bhat))
print("<b|b> =", np.dot(b.conj().T, b))

<bhat|bhat> = (1.0000000000000004+0j)
<b|b> = (1+0j)


# Future Work

* Better optimization methods.
    * Optimize over a subset of the parameters at a time, then loop through (and reoptimize).
    * Add random gates using simulated annealing.
    * Compute all $nK^2$ circuits in parallel.
* Compute expectations of local observables at each cost iteration.
* Allow for arbitrary unitaries (not just Paulis)